In [2]:
cd ..

C:\Users\johan\github\designresolver


In [3]:
from testproblems import generate_random_prob
from representations import bipartite_repr
from inputresolver import resolve,invert_edges,getallvars,direct_eqs
from polycasebuilder import eqv_to_edges_tree
from graphutils import flat_graph_formulation, draw_graph_graphical_props
from operators import sort_scc
#from networkx.algorithms.bipartite import random_graph as bipartite_random_graph

In [4]:
import numpy as np
import networkx as nx
from itertools import chain

In [5]:
rng = np.random.default_rng()

In [251]:
n_eqs = 20
n_vars = 25
seed = 8#8 is triang#seed 10 is nice 42
sparsity = 1.7#0.87#0.8 1.1 #1.7 1.3
inputs=[]
eqv, varinc, dout = generate_random_prob(n_eqs, n_vars, seed, sparsity)
eqvd = {key:tuple(elt for elt in var if elt not in inputs) for key,var in eqv.items()}
#eqv = eqvd
eqs=direct_eqs(eqv, dout)
output_set = {key:var-n_eqs for key,var in dout.items()}

In [252]:
edges, tree = eqv_to_edges_tree(eqv, output_set, n_eqs)

In [253]:
_, edges_original = bipartite_repr(eqv)
avrs = getallvars(eqs, sympy=False)
eqns = eqs.keys()
graph_edges = invert_edges(edges_original)

In [254]:
nx.Graph(({1,2},{3,2})).edges()

EdgeView([(1, 2), (2, 3)])

In [259]:
def dir_graph(undir_edges, rightset, selected):
    # edge order independent
    for node1, node2 in undir_edges:
        if ((node1,node2) in selected or (node2,node1) in selected):
            yield (node2,node1) if node2 in rightset else (node1, node2)
        else:
            yield (node1,node2) if node2 in rightset else (node2, node1)

In [256]:
def heuristic_permute_tear(undir_edges, rightset):
    # edge order independent
    G = nx.Graph(undir_edges)
    G_original = G.copy()
    assignment = tuple()
    vertexelim = set()
    while G.nodes():
        degree = dict(G.degree(rightset))
        mindegarg = min(degree, key=degree.get)
        argneighbors = list(G.neighbors(mindegarg))
        G.remove_node(mindegarg)
        if degree[mindegarg] != 0:
            assignment += ((mindegarg,argneighbors[0]),)
        else:
            vertexelim.add((mindegarg, tuple(G_original.neighbors(mindegarg))))
        for neighbor in argneighbors:
            G.remove_node(neighbor)
    return assignment, vertexelim

In [257]:
rightset = range(n_eqs) #eqs
asg, velim = heuristic_permute_tear(edges_original, rightset)

In [258]:
velim

{(8, (23, 30, 20, 44)),
 (11, (26, 23, 34, 20)),
 (17, (21, 25, 26, 23, 30, 38, 31, 35, 29))}

In [231]:
dedges = dir_graph(graph_edges, asg)

In [181]:
def generate_all_scc(dedges, velim):
    scc_full = set() # make sure we don't run into repeated cycles
    for v, neighbors in velim:
        D = nx.DiGraph(dedges)
        for u in neighbors:
            D.remove_edge(u,v)
            D.add_edge(v,u)
            S = nx.strongly_connected_components(D)
            scc_nonatomic = (elt for elt in S if len(elt)>1)
            for scc in scc_nonatomic:
                scc_full.add(tuple(scc))
    return scc_full

In [183]:
c = generate_all_scc(dedges, velim)

In [221]:
# def generate_all_cycles(dedges, velim):
#     cycles = set() # make sure we don't run into repeated cycles
#     for v, neighbors in velim:
#         D = nx.DiGraph(dedges)
#         for u in neighbors:
#             D.remove_edge(u,v)
#             D.add_edge(v,u)
#             for cycle in nx.simple_cycles(D):
#                 cycles.add(tuple(cycle))
#     return cycles

In [201]:
len(c)

22

In [75]:
dedges

[(0, 3),
 (5, 0),
 (6, 0),
 (7, 0),
 (3, 1),
 (1, 4),
 (7, 1),
 (6, 1),
 (3, 2),
 (4, 2),
 (5, 2),
 (6, 2),
 (7, 2)]

In [124]:
dedges = dir_graph(graph_edges,asg.items())
D = nx.DiGraph(dedges)

In [115]:
dedges

[(0, 3),
 (5, 0),
 (6, 0),
 (7, 0),
 (3, 1),
 (1, 4),
 (7, 1),
 (6, 1),
 (3, 2),
 (4, 2),
 (5, 2),
 (6, 2),
 (7, 2)]

In [95]:
v,u = 2, 3
D.remove_edge(u,v)
D.add_edge(v,u)

In [96]:
list(nx.simple_cycles(nx.DiGraph(D)))

[[1, 4, 2, 3]]

In [ ]:
for elt in asg:
    D = nx.DiGraph(dir_edges)

In [21]:
tuple(asg.items())

((5, 10),
 (0, 20),
 (1, 12),
 (7, 14),
 (9, 13),
 (2, 19),
 (4, 23),
 (3, 16),
 (6, 15))

In [14]:
G = nx.Graph(edges_original)
cntr = 0
new_out = dict()
while G.nodes():
    cntr+=1
    d = dict(G.degree(range(n_eqs)))
    mind = min(d, key=d.get)
    ns = list(G.neighbors(mind))
    G.remove_node(mind)
    if d[mind] != 0:
        first_elt = ns[0]
        new_out[mind] = first_elt
    for elt in ns:
        G.remove_node(elt)

In [15]:
set(range(n_eqs))-new_out.keys()

{8}

In [56]:
def dir_graph(edges, selected):
    dir_edges = [(r,j) if (r,j) in selected else (j,r) for (r,j) in edges]
    return dir_edges

In [57]:
D = nx.DiGraph(dir_graph(graph_edges, new_out.items()))

In [58]:
nx.is_directed_acyclic_graph(D)

True

In [50]:
mind = min(d, key=d.get)

In [51]:
ns = list(G.neighbors(mind))
if d[mind] != 0:
    first_elt = ns[0]
    new_out = {mind: first_elt}
for elt in ns:
    G.remove_node(elt)

In [53]:
new_out

{2: 22}

In [17]:
min(d, key=d.get)

2

In [18]:
d

{0: 4,
 1: 3,
 2: 1,
 3: 7,
 4: 7,
 5: 5,
 6: 3,
 7: 6,
 8: 4,
 9: 4,
 10: 4,
 11: 4,
 12: 6,
 13: 5,
 14: 2,
 15: 8,
 16: 3,
 17: 9,
 18: 5,
 19: 3}